### [How to stream runnables](https://python.langchain.com/docs/how_to/streaming/)

In [1]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. LLMs and Chat Models

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o-mini')

In [4]:
chunks = []
for chunk in model.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end='|', flush=True)

|The| color| of| the| sky| can| vary| depending| on| several| factors|,| including| the| time| of| day|,| weather| conditions|,| and| atmospheric| particles|.| Generally|,| during| a| clear| day|,| the| sky| appears| blue| due| to| Ray|leigh| scattering|,| where| shorter| wavelengths| of| light| are| scattered| more| than| longer| wavelengths|.| At| sunrise| and| sunset|,| the| sky| can| display| shades| of| orange|,| pink|,| and| red| due| to| the| longer| path| of| sunlight| through| the| atmosphere|,| which| scat|ters| shorter| wavelengths| and| allows| longer| wavelengths| to| dominate|.| During| over|cast| or| storm|y| weather|,| the| sky| can| appear| gray|.||

In [5]:
chunks = []
async for chunk in model.astream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end='|', flush=True)

|The| color| of| the| sky| can| vary| depending| on| several| factors|,| including| time| of| day|,| weather|,| and| atmospheric| conditions|.| During| a| clear| day|,| the| sky| typically| appears| blue| due| to| the| scattering| of| sunlight| by| the| atmosphere|.| At| sunrise| and| sunset|,| the| sky| can| display| a| range| of| colors|,| including| shades| of| orange|,| pink|,| purple|,| and| red|.| On| cloudy| or| over|cast| days|,| the| sky| may| appear| gray|.| Additionally|,| pollution| and| other| atmospheric| particles| can| also| affect| the| perceived| color| of| the| sky|.||

In [6]:
chunks[0]

AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-d81da236-025d-4610-9e06-02945965bb0b')

In [7]:
chunks[0] + chunks[1] + chunks[2] + chunks[3] + chunks[4]



AIMessageChunk(content='The color of the', additional_kwargs={}, response_metadata={}, id='run-d81da236-025d-4610-9e06-02945965bb0b')

#### B. Chains

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser


async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

|Why| did| the| par|rot| wear| a| rain|coat|?

|Because| it| wanted| to| be| a| poly|-uns|aturated| bird|!||